In [ ]:
import os
import tensorflow as tf
import numpy as np
from io import open
from tensorboard.plugins import projector

In [ ]:
def create_projection(projection_data,
                      projection_name="tensorboard_viz",
                      path="/tmp/"):
    meta_file = f"{projection_name}.tsv"
    vector_dim = len(projection_data[0][1])
    samples = len(projection_data)
    projection_matrix = np.zeros((samples, vector_dim))

    with open(os.path.join(path, meta_file), 'w') as file_metadata:
         for i, row in enumerate(projection_data):
            label, vector = row[0], row[1]
            projection_matrix[i] = np.array(vector)
            file_metadata.write("{}\n".format(label))

    sess = tf.InteractiveSession()

    embedding = tf.Variable(projection_matrix,
    trainable=False,
    name=projection_name)
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(path, sess.graph)

    config = projector.ProjectorConfig()
    embed = config.embeddings.add()

    embed.tensor_name =  f"{projection_name}"
    embed.metadata_path = os.path.join(path, meta_file)

    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(path, f"{projection_name}.ckpt"))
    print(f"Run `tensorboard --logdir={path}` to run visualize result on tensorboard")

In [ ]:
projection_name = "NLP_in_Action"
projection_data = [
    ('car', [0.34, ..., -0.72]),
    ...
    ('toy', [0.46, ..., 0.39]),
]
create_projection(projection_data, projection_name)